# Ray Tune Example

#### Examples from: 

* https://docs.ray.io/en/latest/tune/examples/tune-sklearn.html
* https://docs.ray.io/en/latest/tune/index.html


In [3]:
import os
from collections import Counter
import platform
import time
import numpy as np

import ray
from ray import tune

from tune_sklearn import TuneGridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV


If we have a ray cluster to connect to, do it. otherwise rely on ray to manage distribution of work locally. 

In [4]:
ray.init('ray://{ray_head}:10001'.format(ray_head=os.environ['RAY_CLUSTER']))

ClientContext(dashboard_url='10.131.2.243:8265', python_version='3.8.12', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', protocol_version='2022-03-16', _num_clients=4, _context_to_restore=<ray.util.client._ClientContext object at 0x7fb0368e1280>)

Just to test this out we will define a simple function for which we want to find the set of parameters that produce lowest score. Here we will use the function `a**2 + b` to define our score. Later on we will use a more complex function, that requires multiple iterations. But for now let's just confirm this works as expected. 

In [5]:
# 1. Define an objective function.
def objective(config, checkpoint_dir=None):
    score = config["a"] ** 2 + config["b"]
    return {"score": score}

We are also going to need to define our search space for our grid search. Again, let's do something super simple so we can evaluate the results by our own inspection and confirm everything is working right. 

In [6]:
# 2. Define a search space.
search_space = {
    "a": tune.grid_search([0.001, 0.01, 0.1]),
    "b": tune.grid_search([1, 2, 3]),
}

Now we invoke our `tune.run` command and see what happens. :) 

We also want to get the results for the set of parameters that give us the smallest score for our objective function. 

In [7]:
# 3. Start a Tune run and print the best result.
analysis = tune.run(objective, config=search_space,resources_per_trial={'gpu': 1})
print(analysis.get_best_config(metric="score", mode="min"))

(run pid=2682) 2022-07-22 17:18:44,994	INFO trial_runner.py:803 -- starting objective_562f1_00000
(run pid=2682) 2022-07-22 17:18:45,023	ERROR syncer.py:119 -- Log sync requires rsync to be installed.


(run pid=2682) == Status ==
(run pid=2682) Current time: 2022-07-22 17:18:46 (running for 00:00:03.15)
(run pid=2682) Memory usage on this node: 8.1/30.9 GiB
(run pid=2682) Using FIFO scheduling algorithm.
(run pid=2682) Resources requested: 1.0/1 CPUs, 1.0/1 GPUs, 0.0/2.8 GiB heap, 0.0/1.13 GiB objects
(run pid=2682) Result logdir: /home/ray_results/objective_2022-07-22_17-18-43
(run pid=2682) Number of trials: 9/9 (8 PENDING, 1 RUNNING)
(run pid=2682) +-----------------------+----------+-------------------+-------+-----+
(run pid=2682) | Trial name            | status   | loc               |     a |   b |
(run pid=2682) |-----------------------+----------+-------------------+-------+-----|
(run pid=2682) | objective_562f1_00000 | RUNNING  | 10.131.2.243:2788 | 0.001 |   1 |
(run pid=2682) | objective_562f1_00001 | PENDING  |                   | 0.01  |   1 |
(run pid=2682) | objective_562f1_00002 | PENDING  |                   | 0.1   |   1 |
(run pid=2682) | objective_562f1_00003 | 

(run pid=2682) 2022-07-22 17:18:48,417	INFO trial_runner.py:803 -- starting objective_562f1_00001


(run pid=2682) Result for objective_562f1_00001:
(run pid=2682)   date: 2022-07-22_17-18-50
(run pid=2682)   done: false
(run pid=2682)   experiment_id: d0e73a616fff43dbaea9389c68d02060
(run pid=2682)   hostname: ray-cluster-mcliffor-40redhat-2ecom-ray-cluster-mcliffor-499pj7
(run pid=2682)   iterations_since_restore: 1
(run pid=2682)   node_ip: 10.131.2.243
(run pid=2682)   pid: 2891
(run pid=2682)   score: 1.0001
(run pid=2682)   time_since_restore: 0.00014495849609375
(run pid=2682)   time_this_iter_s: 0.00014495849609375
(run pid=2682)   time_total_s: 0.00014495849609375
(run pid=2682)   timestamp: 1658510330
(run pid=2682)   timesteps_since_restore: 0
(run pid=2682)   training_iteration: 1
(run pid=2682)   trial_id: 562f1_00001
(run pid=2682)   warmup_time: 0.002509593963623047
(run pid=2682)   
(run pid=2682) == Status ==
(run pid=2682) Current time: 2022-07-22 17:18:50 (running for 00:00:06.56)
(run pid=2682) Memory usage on this node: 8.2/30.9 GiB
(run pid=2682) Using FIFO sche

(run pid=2682) 2022-07-22 17:18:52,127	INFO trial_runner.py:803 -- starting objective_562f1_00002


...

(run pid=2682) 2022-07-22 17:18:59,166	INFO trial_runner.py:803 -- starting objective_562f1_00004


(run pid=2682) Result for objective_562f1_00004:
(run pid=2682)   date: 2022-07-22_17-19-00
(run pid=2682)   done: false
(run pid=2682)   experiment_id: fb63af60920d4018a57bbe4f2e94c699
(run pid=2682)   hostname: ray-cluster-mcliffor-40redhat-2ecom-ray-cluster-mcliffor-499pj7
(run pid=2682)   iterations_since_restore: 1
(run pid=2682)   node_ip: 10.131.2.243
(run pid=2682)   pid: 3204
(run pid=2682)   score: 2.0001
(run pid=2682)   time_since_restore: 0.00015997886657714844
(run pid=2682)   time_this_iter_s: 0.00015997886657714844
(run pid=2682)   time_total_s: 0.00015997886657714844
(run pid=2682)   timestamp: 1658510340
(run pid=2682)   timesteps_since_restore: 0
(run pid=2682)   training_iteration: 1
(run pid=2682)   trial_id: 562f1_00004
(run pid=2682)   warmup_time: 0.0025877952575683594
(run pid=2682)   
(run pid=2682) == Status ==
(run pid=2682) Current time: 2022-07-22 17:19:00 (running for 00:00:17.33)
(run pid=2682) Memory usage on this node: 8.2/30.9 GiB
(run pid=2682) Using

(run pid=2682) 2022-07-22 17:19:03,391	INFO trial_runner.py:803 -- starting objective_562f1_00005
(run pid=2682) 2022-07-22 17:19:03,530	INFO trial_runner.py:803 -- starting objective_562f1_00006
(run pid=2682) 2022-07-22 17:19:04,185	INFO trial_runner.py:803 -- starting objective_562f1_00007
(run pid=2682) 2022-07-22 17:19:04,219	INFO trial_runner.py:803 -- starting objective_562f1_00008


(run pid=2682) Result for objective_562f1_00005:
(run pid=2682)   date: 2022-07-22_17-19-04
(run pid=2682)   done: false
(run pid=2682)   experiment_id: 07942328e00341f09d01a85c5ad0e866
(run pid=2682)   hostname: ray-cluster-mcliffor-40redhat-2ecom-ray-cluster-mcliffor-4prdz4
(run pid=2682)   iterations_since_restore: 1
(run pid=2682)   node_ip: 10.128.4.73
(run pid=2682)   pid: 202
(run pid=2682)   score: 2.01
(run pid=2682)   time_since_restore: 0.0001308917999267578
(run pid=2682)   time_this_iter_s: 0.0001308917999267578
(run pid=2682)   time_total_s: 0.0001308917999267578
(run pid=2682)   timestamp: 1658510344
(run pid=2682)   timesteps_since_restore: 0
(run pid=2682)   training_iteration: 1
(run pid=2682)   trial_id: 562f1_00005
(run pid=2682)   warmup_time: 0.0030412673950195312
(run pid=2682)   
(run pid=2682) Result for objective_562f1_00005:
(run pid=2682)   date: 2022-07-22_17-19-04
(run pid=2682)   done: true
(run pid=2682)   experiment_id: 07942328e00341f09d01a85c5ad0e866


(run pid=2682) 2022-07-22 17:19:06,327	INFO tune.py:701 -- Total run time: 23.28 seconds (22.60 seconds for the tuning loop).


Cool, It works! At least, the ray tune tool doesn't break. Now let's 1) apply it to a more complex function and 2) compare it to the performance of vanilla sklearn. First thing we need to do is create a dataset and partition it into training and testing sets for evaluation.   




In [8]:
# Create dataset
X, y = make_classification(
    n_samples=11000,
    n_features=1000,
    n_informative=50,
    n_redundant=0,
    n_classes=10,
    class_sep=2.5,
)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1000)

This function has different parameters the the one above, so let's define our new search space for the SGD (Stochastic Gradient Descent) algorithm. 

In [9]:
# 2. Define a search space.
search_space = {
    "alpha": tune.grid_search([1e-4, 1e-1, 1]),
    "epsilon": tune.grid_search([0.01, 0.1]),
}

As a baseline to evaluate our increased performance. Let's run SGD once, with one set of parameters and see how long it takes. 

In [10]:
clf = SGDClassifier(alpha=1,epsilon=0.1)
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

0.889

Looks like it takes about 1 second to initialize the SGD, train it and evaluate it. Give our search space contains 6 different combinations, we should expect a serial execution (slowest) to be about 6 seconds, and a fully parralelized implementation to take about 1 second (fastest). Lets see.    

In [11]:
def objective(config, checkpoint_dir=None):
    clf = SGDClassifier(alpha = config["alpha"], epsilon = config["epsilon"])
    clf.fit(x_train,y_train)
    score = clf.score(x_test,y_test)
    return {"score": score}

In [12]:
analysis = tune.run(objective, config=search_space,resources_per_trial={'gpu': 1})
print(analysis.get_best_config(metric="score", mode="max"))

2022-07-22 17:19:47,061 [13487] WARNING  py.warnings: /opt/app-root/lib64/python3.8/site-packages/ray/util/client/worker.py:599: UserWarning: More than 10MB of messages have been created to schedule tasks on the server. This can be slow on Ray Client due to communication overhead over the network. If you're running many fine-grained tasks, consider running them inside a single remote function. See the section on "Too fine-grained tasks" in the Ray Design Patterns document for more details: https://docs.google.com/document/d/167rnnDFIVRhHhK4mznEIemOtj63IOhtIPvSYaPgI4Fg/edit#heading=h.f7ins22n6nyl. If your functions frequently use large objects, consider storing the objects remotely with ray.put. An example of this is shown in the "Closure capture of large / unserializable object" section of the Ray Design Patterns document, available here: https://docs.google.com/document/d/167rnnDFIVRhHhK4mznEIemOtj63IOhtIPvSYaPgI4Fg/edit#heading=h.1afmymq455wu
  warnings.warn(

(run pid=2682) 2022-07-

(run pid=2682) == Status ==
(run pid=2682) Current time: 2022-07-22 17:20:05 (running for 00:00:14.54)
(run pid=2682) Memory usage on this node: 8.7/30.9 GiB
(run pid=2682) Using FIFO scheduling algorithm.
(run pid=2682) Resources requested: 1.0/4 CPUs, 1.0/4 GPUs, 0.0/11.2 GiB heap, 0.0/4.66 GiB objects
(run pid=2682) Result logdir: /home/ray_results/objective_2022-07-22_17-19-50
(run pid=2682) Number of trials: 6/6 (5 PENDING, 1 RUNNING)
(run pid=2682) +-----------------------+----------+------------------+---------+-----------+
(run pid=2682) | Trial name            | status   | loc              |   alpha |   epsilon |
(run pid=2682) |-----------------------+----------+------------------+---------+-----------|
(run pid=2682) | objective_7e5c3_00000 | RUNNING  | 10.131.0.141:263 |  0.0001 |      0.01 |
(run pid=2682) | objective_7e5c3_00001 | PENDING  |                  |  0.1    |      0.01 |
(run pid=2682) | objective_7e5c3_00002 | PENDING  |                  |  1      |      0.01 

(run pid=2682) 2022-07-22 17:20:05,108	INFO trial_runner.py:803 -- starting objective_7e5c3_00001
(run pid=2682) 2022-07-22 17:20:06,010	WARNING util.py:171 -- The `start_trial` operation took 0.902 s, which may be a performance bottleneck.
(run pid=2682) 2022-07-22 17:20:06,011	INFO trial_runner.py:803 -- starting objective_7e5c3_00002
(run pid=2682) 2022-07-22 17:20:07,911	WARNING util.py:171 -- The `start_trial` operation took 1.900 s, which may be a performance bottleneck.


(run pid=2682) == Status ==
(run pid=2682) Current time: 2022-07-22 17:20:10 (running for 00:00:20.09)
(run pid=2682) Memory usage on this node: 9.0/30.9 GiB
(run pid=2682) Using FIFO scheduling algorithm.
(run pid=2682) Resources requested: 3.0/4 CPUs, 3.0/4 GPUs, 0.0/11.2 GiB heap, 0.0/4.66 GiB objects
(run pid=2682) Result logdir: /home/ray_results/objective_2022-07-22_17-19-50
(run pid=2682) Number of trials: 6/6 (3 PENDING, 3 RUNNING)
(run pid=2682) +-----------------------+----------+-------------------+---------+-----------+
(run pid=2682) | Trial name            | status   | loc               |   alpha |   epsilon |
(run pid=2682) |-----------------------+----------+-------------------+---------+-----------|
(run pid=2682) | objective_7e5c3_00000 | RUNNING  | 10.131.0.141:263  |  0.0001 |      0.01 |
(run pid=2682) | objective_7e5c3_00001 | RUNNING  | 10.131.2.243:3660 |  0.1    |      0.01 |
(run pid=2682) | objective_7e5c3_00002 | RUNNING  | 10.128.4.73:263   |  1      |     

(run pid=2682) 2022-07-22 17:20:10,653	INFO trial_runner.py:803 -- starting objective_7e5c3_00003


...

(run pid=2682) 2022-07-22 17:20:15,119	INFO trial_runner.py:803 -- starting objective_7e5c3_00004


(run pid=2682) Result for objective_7e5c3_00001:
(run pid=2682)   date: 2022-07-22_17-20-13
(run pid=2682)   done: true
(run pid=2682)   experiment_id: 6d6b52e1ada146de8472b8b35ca17322
(run pid=2682)   experiment_tag: 1_alpha=0.1,epsilon=0.01
(run pid=2682)   hostname: ray-cluster-mcliffor-40redhat-2ecom-ray-cluster-mcliffor-499pj7
(run pid=2682)   iterations_since_restore: 1
(run pid=2682)   node_ip: 10.131.2.243
(run pid=2682)   pid: 3660
(run pid=2682)   score: 0.894
(run pid=2682)   time_since_restore: 3.877516269683838
(run pid=2682)   time_this_iter_s: 3.877516269683838
(run pid=2682)   time_total_s: 3.877516269683838
(run pid=2682)   timestamp: 1658510413
(run pid=2682)   timesteps_since_restore: 0
(run pid=2682)   training_iteration: 1
(run pid=2682)   trial_id: 7e5c3_00001
(run pid=2682)   warmup_time: 0.0027153491973876953
(run pid=2682)   


(run pid=2682) 2022-07-22 17:20:15,988	WARNING util.py:171 -- The `start_trial` operation took 0.869 s, which may be a performance bottleneck.
(run pid=2682) 2022-07-22 17:20:18,427	INFO trial_runner.py:803 -- starting objective_7e5c3_00005
(run pid=2682) 2022-07-22 17:20:19,340	WARNING util.py:171 -- The `start_trial` operation took 0.913 s, which may be a performance bottleneck.


(run pid=2682) == Status ==
(run pid=2682) Current time: 2022-07-22 17:20:22 (running for 00:00:31.60)
(run pid=2682) Memory usage on this node: 9.1/30.9 GiB
(run pid=2682) Using FIFO scheduling algorithm.
(run pid=2682) Resources requested: 4.0/4 CPUs, 4.0/4 GPUs, 0.0/11.2 GiB heap, 0.0/4.66 GiB objects
(run pid=2682) Result logdir: /home/ray_results/objective_2022-07-22_17-19-50
(run pid=2682) Number of trials: 6/6 (4 RUNNING, 2 TERMINATED)
(run pid=2682) +-----------------------+------------+-------------------+---------+-----------+--------+------------------+---------+
(run pid=2682) | Trial name            | status     | loc               |   alpha |   epsilon |   iter |   total time (s) |   score |
(run pid=2682) |-----------------------+------------+-------------------+---------+-----------+--------+------------------+---------|
(run pid=2682) | objective_7e5c3_00000 | RUNNING    | 10.131.0.141:263  |  0.0001 |      0.01 |        |                  |         |
(run pid=2682) | 

(run pid=2682) 2022-07-22 17:20:45,116	INFO tune.py:701 -- Total run time: 56.23 seconds (54.45 seconds for the tuning loop).


In [13]:
# Example parameters to tune from SGDClassifier
parameter_grid = {"alpha": [1e-4, 1e-1, 1], "epsilon": [0.01, 0.1]}

# n_jobs=-1 enables use of all cores like Tune does
sklearn_search = GridSearchCV(SGDClassifier(), parameter_grid, n_jobs=-1)

start = time.time()
sklearn_search.fit(x_train, y_train)
end = time.time()
print("Sklearn Fit Time:", end - start)

Sklearn Fit Time: 37.59654688835144


#### Note: This is actually using a different method of using ray tune 

In [14]:
tune_search = TuneGridSearchCV(
    SGDClassifier(), parameter_grid, early_stopping=True, max_iters=10
)

start = time.time()
tune_search.fit(x_train, y_train)
end = time.time()
print("Tune GridSearch Fit Time:", end - start)
# Tune GridSearch Fit Time: 15.436315774917603 (for an 8 core laptop)

(run pid=2682) 2022-07-22 17:24:03,657	INFO trial_runner.py:803 -- starting _Trainable_1465f_00000
(run pid=2682) 2022-07-22 17:24:03,726	INFO trial_runner.py:803 -- starting _Trainable_1465f_00001
(run pid=2682) 2022-07-22 17:24:03,731	INFO trial_runner.py:803 -- starting _Trainable_1465f_00002
(run pid=2682) 2022-07-22 17:24:03,860	INFO trial_runner.py:803 -- starting _Trainable_1465f_00003
(run pid=2682) 2022-07-22 17:24:08,126	INFO trial_runner.py:803 -- starting _Trainable_1465f_00004
(run pid=2682) 2022-07-22 17:24:15,609	INFO trial_runner.py:803 -- starting _Trainable_1465f_00005


Tune GridSearch Fit Time: 19.755069732666016


In [15]:
# n_jobs=-1 enables use of all cores like Tune does
sklearn_search = GridSearchCV(SGDClassifier(), parameter_grid, n_jobs=-1)

start = time.time()
sklearn_search.fit(x_train, y_train)
end = time.time()
print("Sklearn Fit Time:", end - start)


Sklearn Fit Time: 39.39723062515259
